## MCP Primitives
================

### Tools

In [1]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Primitives Demo")

@mcp.tool()
def searchFlights(origin: str, destination: str, date: str) -> dict:
    """Search for available flights (YYYY-MM-DD)."""
    # Demo estático (en real, aquí llamarías a tu API de vuelos)
    return {
        "flights": [
            {"flight_number": "AA123", "departure": origin, "arrival": destination, "date": date, "price": 300},
            {"flight_number": "DL456", "departure": origin, "arrival": destination, "date": date, "price": 350},
        ]
    }


### Resources

In [3]:
@mcp.resource(
    uri = "travel://flights/{origin}/{destination}",
    name = "flight-search",
    title = "Flight Search",
    description = "Search available flights between cities",
    mime_type="application/json",
)
def flight_search(origin: str, destination: str) -> dict:
    """Search Flights between origin and destination"""
    return {
        "flights": [
            {"flight_number": "AA123", "departure": origin, "arrival": destination, "price": 300},
            {"flight_number": "DL456", "departure": origin, "arrival": destination, "price": 350},
        ]   
    }

### Prompts

In [4]:
@mcp.prompt(title="Plan a vacation", description="Guide through vacation planning process")
def planVacation(destination: str, duration: int, budget: float, interests: list[str]) -> str:
    return (
        f"- Planning a {duration}-day vacation to {destination} with a budget of ${budget:.0f}.\n"
        f"  Interests include: {', '.join(interests)}."
    )


### OpenAI and Azure MCP server

In [2]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
import json, os, logging, asyncio
from dotenv import load_dotenv

In [3]:

# Setup logging and load environment variables
logger = logging.getLogger(__name__)

In [4]:
# Azure OpenAI configuration

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")

# Initialize Azure credentials
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

In [5]:
# Initialize Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT, 
        api_version="2024-04-01-preview", 
        azure_ad_token_provider=token_provider
    )